<a href="https://colab.research.google.com/github/Tanzaniav0825/CS667/blob/main/Project_1_deliverables_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
SERPAPI_KEY = getpass("Enter your SerpAPI key:")

Enter your SerpAPI key:··········


In [8]:
!pip install tldextract
!pip install joblib



In [13]:
!pip install google-search-results


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=d136ec3d795a496165cd36d365d4c85df9e0adad824accfd8370bc664a021e18
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [19]:
import tldextract
import re
import random
from serpapi import GoogleSearch

class CredibilityScorer:
    def __init__(self, url, serpapi_key=None):
        self.url = url
        self.serpapi_key = serpapi_key
        self.domain_info = tldextract.extract(url)
        self.domain = f"{self.domain_info.domain}.{self.domain_info.suffix}"
        self.score = 0.0
        self.explanation_parts = []

    def domain_suffix_score(self):
        suffix = self.domain_info.suffix
        if suffix in ["edu", "gov"]:
            self.score += 0.3
            self.explanation_parts.append(f"✅ Domain suffix '.{suffix}' indicates high credibility (academic/government).")
        elif suffix == "org":
            self.score += 0.2
            self.explanation_parts.append("🟡 '.org' is often non-profit, moderately credible.")
        else:
            self.score += 0.1
            self.explanation_parts.append(f"⚠️ '.{suffix}' is a common/commercial domain — minimal trust boost.")

    def trusted_domain_score(self):
        trusted_sources = ["nature.com", "sciencedirect.com", "springer.com", "jstor.org", "arxiv.org"]
        if any(source in self.url for source in trusted_sources):
            self.score += 0.4
            self.explanation_parts.append(f"✅ Trusted academic publisher detected: `{self.domain}`.")
        else:
            self.explanation_parts.append(f"🔍 `{self.domain}` not on the trusted source list.")

    def recent_year_score(self):
        if re.search(r"/20\d{2}/", self.url):
            self.score += 0.1
            self.explanation_parts.append("📅 Recent year found in URL — likely up-to-date.")
        else:
            self.explanation_parts.append("📅 No recent date detected in URL.")

    def https_check(self):
        if self.url.startswith("https://"):
            self.score += 0.1
            self.explanation_parts.append("🔒 Secure HTTPS protocol detected.")
        else:
            self.explanation_parts.append("⚠️ URL uses unsecure HTTP protocol.")

    def url_length_score(self):
        if len(self.url) < 75:
            self.score += 0.1
            self.explanation_parts.append("📏 Short, clean URL structure.")
        elif len(self.url) < 150:
            self.score += 0.05
            self.explanation_parts.append("📏 Moderate-length URL.")
        else:
            self.explanation_parts.append("⚠️ Long or messy URL — potential spam signal.")

    def keyword_check(self):
        keywords = ["study", "journal", "article", "research", "doi"]
        if any(word in self.url.lower() for word in keywords):
            self.score += 0.1
            self.explanation_parts.append("🔍 Scholarly keywords found in URL path.")

    def spam_word_check(self):
        spam_words = ["shocking", "miracle", "secret", "click", "you-won’t-believe"]
        if any(word in self.url.lower() for word in spam_words):
            self.score -= 0.2
            self.explanation_parts.append("🚫 Spammy/clickbait keywords found — credibility penalized.")

    def ml_model_score(self):
        ml_score = round(random.uniform(0.6, 0.95), 2)
        self.score += ml_score * 0.2
        self.explanation_parts.append(f"🤖 Simulated ML model credibility score = {ml_score}.")

    def serpapi_credibility_score(self):
        if not self.serpapi_key:
            self.explanation_parts.append("🔍 SERP analysis skipped (no API key provided).")
            return

        try:
            params = {
                "q": self.url,
                "api_key": self.serpapi_key,
                "num": 5
            }
            search = GoogleSearch(params)
            results = search.get_dict()
            found = False

            for idx, result in enumerate(results.get("organic_results", [])):
                serp_url = result.get("link", "")
                serp_domain = tldextract.extract(serp_url)
                serp_normalized = f"{serp_domain.domain}.{serp_domain.suffix}"

                if serp_normalized == self.domain:
                    found = True
                    if idx == 0:
                        self.score += 0.2
                        self.explanation_parts.append("🔝 URL appears as the top Google result.")
                    elif idx <= 2:
                        self.score += 0.1
                        self.explanation_parts.append("⬆️ URL ranks within top 3 Google results.")
                    else:
                        self.explanation_parts.append("🔍 URL found in lower Google results.")

                    title = result.get("title", "").lower()
                    snippet = result.get("snippet", "").lower()
                    if any(kw in title + snippet for kw in ["study", "doi", "research", "journal"]):
                        self.score += 0.1
                        self.explanation_parts.append("📚 SERP snippet contains scholarly language.")
                    if any(kw in title for kw in ["miracle", "shocking", "click here"]):
                        self.score -= 0.2
                        self.explanation_parts.append("🚫 Clickbait detected in SERP title.")
                    break

            if not found:
                self.explanation_parts.append("⚠️ Domain not matched in top Google search results.")

        except Exception as e:
            self.explanation_parts.append(f"⚠️ SERP fetch failed: {str(e)}")

    def compute_star_rating(self):
        if self.score >= 0.9:
            return "⭐⭐⭐⭐⭐"
        elif self.score >= 0.75:
            return "⭐⭐⭐⭐"
        elif self.score >= 0.6:
            return "⭐⭐⭐"
        elif self.score >= 0.4:
            return "⭐⭐"
        else:
            return "⭐"

    def evaluate(self):
        self.domain_suffix_score()
        self.trusted_domain_score()
        self.recent_year_score()
        self.https_check()
        self.url_length_score()
        self.keyword_check()
        self.spam_word_check()
        self.ml_model_score()
        self.serpapi_credibility_score()

        return {
            "url": self.url,
            "score": round(min(self.score, 1.0), 2),
            "stars": self.compute_star_rating(),
            "explanation": " ".join(self.explanation_parts)
        }

# -------------------------------
# 🧪 User Input Test
# -------------------------------
if __name__ == "__main__":
    from getpass import getpass

    print("🔍 Credibility Scorer")
    input_url = input("Paste a URL to evaluate: ").strip()
    serp_key = getpass("🔐 Enter your SerpAPI key (press Enter to skip): ").strip()

    if not input_url.startswith("http"):
        print("⚠️ Please enter a valid URL with http:// or https://")
    else:
        scorer = CredibilityScorer(input_url, serpapi_key=serp_key if serp_key else None)
        result = scorer.evaluate()

        print("\n🎯 Evaluation Result")
        print(f"🔗 URL: {result['url']}")
        print(f"⭐ Score: {result['stars']} ({result['score']} / 1.0)")
        print("📘 Explanation:")
        print(result["explanation"])


🔍 Credibility Scorer
Paste a URL to evaluate: https://www.nature.com/articles/s41586-020-03119-7
🔐 Enter your SerpAPI key (press Enter to skip): ··········

🎯 Evaluation Result
🔗 URL: https://www.nature.com/articles/s41586-020-03119-7
⭐ Score: ⭐⭐⭐⭐⭐ (0.94 / 1.0)
📘 Explanation:
⚠️ '.com' is a common/commercial domain — minimal trust boost. ✅ Trusted academic publisher detected: `nature.com`. 📅 No recent date detected in URL. 🔒 Secure HTTPS protocol detected. 📏 Short, clean URL structure. 🔍 Scholarly keywords found in URL path. 🤖 Simulated ML model credibility score = 0.68. ⚠️ Domain not matched in top Google search results.
